In [25]:
import pandas as pd
import numpy as np
from tools import seq_to_num, acc_score

In [2]:
df_train = pd.read_csv("../data/train.csv", index_col=0)
df_test = pd.read_csv("../data/test.csv", index_col=0)

In [137]:
train_bins = seq_to_num(df_train.Sequence, target_split=True,
                        pad_maxlen=100, dtype=np.float32, drop_na_inf=True,
                        nbins=5, bins_by='terms')
test_bins = seq_to_num(df_test.Sequence, target_split=True, pad_maxlen=100,
                            dtype=np.float32, drop_na_inf=True, nbins=5, bins_by='terms')

In [149]:
train_X, train_y = train_bins[4]

In [150]:
test_X, test_y = test_bins[4]

In [151]:
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor

In [152]:
dt = DecisionTreeRegressor(random_state=42)
dt.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [153]:
acc_score(dt.predict(test_X), test_y)

0.09809663250366032

In [154]:
train_X2, train_y2 = train_bins[1]
test_X2, test_y2 = test_bins[1]

In [155]:
etr = ExtraTreeRegressor(max_depth=100, random_state=42)
etr.fit(train_X2, train_y2)

/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


ExtraTreeRegressor(criterion='mse', max_depth=100, max_features='auto',
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=42, splitter='random')

In [157]:
acc_score(etr.predict(test_X2), test_y2)

0.0017626321974148062

In [158]:
# too long sequence?
train_X2, train_y2 = train_bins[2]
test_X2, test_y2 = test_bins[2]
etr = DecisionTreeRegressor(max_depth=5, random_state=42)
etr.fit(train_X2, train_y2)
acc_score(etr.predict(test_X2), test_y2)

0.0

In [159]:
from sklearn.neural_network import MLPRegressor

In [160]:
# NNet still doesn't work
mlp = MLPRegressor(hidden_layer_sizes=(10, 1))
mlp.fit(train_X, train_y)
acc_score(mlp.predict(test_X), test_y)

/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.0

In [162]:
# Try to combine predictions for bin 3 and 4 (by terms), while
# fallback to mode on bin 0, 1, 2
def mmode(arr):
    modes = []
    for row in arr:
        counts = {i: row.tolist().count(i) for i in row}
        if len(counts) > 0:
            modes.append(max(counts.items(), key=lambda x:x[1])[0])
        else:
            modes.append(0)
    return modes

In [163]:
kg_train = pd.read_csv('../data/kaggle_train.csv', index_col=0)
kg_test = pd.read_csv('../data/kaggle_test.csv', index_col=0)

In [268]:
train_bins = seq_to_num(kg_train.Sequence, target_split=True,
                        pad_maxlen=100, dtype=np.float64, drop_na_inf=True,
                        nbins=5, bins_by='terms')
test_bins = seq_to_num(kg_test.Sequence, target_split=False, pad_maxlen=99,
                            dtype=np.float64, drop_na_inf=False, nbins=5, bins_by='terms')

In [165]:
dt_bin3 = DecisionTreeRegressor(random_state=42)
dt_bin4 = DecisionTreeRegressor(random_state=42)
dt_bin3.fit(*train_bins[3])
dt_bin4.fit(*train_bins[4])

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [170]:
pred_bin3 = dt_bin3.predict(test_bins[3])
pred_bin4 = dt_bin4.predict(test_bins[4])

In [173]:
pred_bin0 = mmode(test_bins[0])

In [174]:
pred_bin1 = mmode(test_bins[1])
pred_bin2 = mmode(test_bins[2])

In [278]:
pd.Series(pred_bin3, index=test_bins[3][0])

ValueError: Length of passed values is 22764, index implies 22769